In [1]:
import numpy as np
import pandas as pd
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

In [2]:
raw_data = pd.read_csv("IPL_Ball_by_Ball_2008_2022.csv")
raw_data = raw_data.sample(frac=0.025, random_state=42)
raw_data

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
177814,501226,1,12,2,MS Dhoni,Yuvraj Singh,MEK Hussey,NaN,1,0,1,0,0,NaN,NaN,NaN,Chennai Super Kings
3287,1304107,1,17,4,SW Billings,T Natarajan,AD Russell,NaN,6,0,6,0,0,NaN,NaN,NaN,Kolkata Knight Riders
188612,419148,2,13,3,RV Uthappa,AD Mathews,LRPL Taylor,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
96212,980961,1,13,6,RA Jadeja,Z Khan,KD Karthik,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Lions
30637,1254064,1,5,1,RR Pant,JD Unadkat,AM Rahane,NaN,4,0,4,0,0,NaN,NaN,NaN,Delhi Capitals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95356,980969,2,14,1,V Kohli,R Bhatia,SR Watson,NaN,4,0,4,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
216575,336022,1,8,2,V Sehwag,RR Powar,S Dhawan,wides,0,1,1,0,0,NaN,NaN,NaN,Delhi Daredevils
16561,1304052,1,18,5,UT Yadav,Akash Deep,CV Varun,NaN,0,0,0,0,1,UT Yadav,bowled,NaN,Kolkata Knight Riders
8938,1304084,2,13,4,AT Rayudu,Sandeep Sharma,RA Jadeja,NaN,4,0,4,0,0,NaN,NaN,NaN,Chennai Super Kings


In [3]:
def over_data(raw_data):
    
    raw_data=raw_data[raw_data['total_run']<=6] # Outliers
    
    over_info = pd.DataFrame()
    
    over_info['overid'] = raw_data['ID']*40+(raw_data['innings']-1)*20+raw_data['overs'] # Unique ID for over
    over_info['type'] = (raw_data['overs']>6).astype(int) + (raw_data['overs']>15).astype(int) # Powerplay/Middle/Death
    
    # Batter Columns
    distinct_batters = raw_data['batter'].unique()
    batter_names = pd.concat([(raw_data['batter'].fillna('').isin([batter]) | raw_data['non-striker'].fillna('').isin([batter])).astype(int) for batter in distinct_batters], axis=1)
    batter_names.columns = distinct_batters
    over_info = pd.concat([over_info,batter_names],axis=1)

    # Bowler Columns
    distinct_bowlers = raw_data['bowler'].unique()
    bowler_names = pd.concat([raw_data['bowler'].fillna('') == bowler for bowler in distinct_bowlers], axis=1).astype(int)
    bowler_names.columns = distinct_bowlers
    over_info = pd.concat([over_info,bowler_names],axis=1)
    
    over_info['total_run'] = raw_data['total_run'] 
    over_info['total_wickets'] = raw_data['isWicketDelivery']
    
    # Grouping each over's information
    over_info = over_info.groupby(over_info['overid']).sum()
    for batter in distinct_batters :
        over_info[batter] = (over_info[batter]>0).astype(int)
    for bowler in distinct_bowlers :
        over_info[bowler]=(over_info[bowler]>0).astype(int)
    
    return over_info

In [10]:
modified_data = over_data(raw_data)
modified_data

,type,MS Dhoni,SW Billings,RV Uthappa,RA Jadeja,RR Pant,KL Rahul,YK Pathan,E Lewis,TM Dilshan,...,Abdur Razzak,GHS Garton,SD Chitnis,VS Yeligati,MB Parmar,A Choudhary,PH Solanki,IC Porel,total_run,total_wickets
overid,,,,,,,,,,,,,,,,,,,,,
13439280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13439282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13439287,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13439288,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13439289,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52488005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
52488019,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
52488020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = modified_data.drop(columns = ["total_run","total_wickets"])
Y = modified_data["total_run"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 1092003)

In [12]:
def Logistic_Regression(X_train,Y_train):
    model = LogisticRegression(n_jobs=-1)
    model.fit(X_train,Y_train)
    return model

In [13]:
def SVM(X_train,Y_train):
    model = SVC(probability=True)
    model.fit(X_train,Y_train)
    return model

In [14]:
def MLP(X_train,Y_train):
    model = MLPClassifier(activation = 'relu',hidden_layer_sizes=(10,5,10), max_iter=30000)
    model.fit(X_train,Y_train)
    return model

In [15]:
print("Using Logistic Regression :-")
model_lr = Logistic_Regression(X_train,Y_train)
Y_train_pred = model_lr.predict(X_train)
Y_test_pred = model_lr.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using Logistic Regression :-


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

Accuracy on Trained data -  0.548968460991226
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.55      0.60      0.58      1415
           1       0.54      0.77      0.64      1750
           2       0.64      0.12      0.20       278
           3       0.00      0.00      0.00        28
           4       0.66      0.12      0.20       476
           5       0.00      0.00      0.00        34
           6       0.92      0.05      0.10       208
           7       1.00      0.06      0.12        16
           8       1.00      0.25      0.40         4
           9       0.00      0.00      0.00         1
          10       1.00      0.17      0.29         6
          12       0.00      0.00      0.00         1

    accuracy                           0.55      4217
   macro avg       0.53      0.18      0.21      4217
weighted avg       0.58      0.55      0.50      4217

Confusion matrix on Trained data - 
 [[ 856  

In [16]:
print("Using SVM :-")
model_SVM = SVM(X_train,Y_train)
Y_train_pred = model_SVM.predict(X_train)
Y_test_pred = model_SVM.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using SVM :-
Accuracy on Trained data -  0.5778989803177614
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.61      0.63      0.62      1415
           1       0.56      0.86      0.68      1750
           2       0.54      0.07      0.13       278
           3       0.00      0.00      0.00        28
           4       1.00      0.03      0.05       476
           5       1.00      0.03      0.06        34
           6       0.00      0.00      0.00       208
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         6
          12       0.00      0.00      0.00         1

    accuracy                           0.58      4217
   macro avg       0.31      0.14      0.13      4217
weighted avg       0.59      0.58      0.50      4217

Confusion matrix on Trained dat

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metr

In [17]:
print("Using MLP :-")
model_MLP = MLP(X_train,Y_train)
Y_train_pred = model_MLP.predict(X_train)
Y_test_pred = model_MLP.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using MLP :-
Accuracy on Trained data -  0.8807208916291203
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.93      0.90      0.91      1415
           1       0.93      0.94      0.94      1750
           2       0.68      0.85      0.76       278
           3       0.00      0.00      0.00        28
           4       0.80      0.90      0.84       476
           5       0.00      0.00      0.00        34
           6       0.64      0.63      0.64       208
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         6
          12       0.00      0.00      0.00         1

    accuracy                           0.88      4217
   macro avg       0.33      0.35      0.34      4217
weighted avg       0.87      0.88      0.87      4217

Confusion matrix on Trained dat

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metr